THU THẬP DỮ LIỆU
##Câu hỏi đặt ra: "Khi nào thì mưa????"


In [14]:
import requests
import json
import csv
import pandas as pd


In [2]:
# request lấy dữ liệu 1 năm 12x30 ngày
# ngày 1-> 28 mỗi tháng, tại vì em lười handle 
# các cột: Ngày + giờ, tempC, Windspeed(km/h),winddir16Point,weatherCode,humidity,visibility(km),pressure,cloudcover


elements = []
for i in range(1,12):
    month = i
    url = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-'+str(month)+'-01&enddate=2013-'+str(month)+'-28&includelocation=yes'

    print(f'GET {url}')
    r = requests.get(url)
    json_pydata = json.loads(r.text)
    for day in range(0,27):

        for j in range(0,7):
            element = []
            element.append(json_pydata['data']['weather'][day]['date']+' - ' +json_pydata['data']['weather'][day]['hourly'][j]['time'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['tempC'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['windspeedKmph'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['winddir16Point'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['weatherCode'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['humidity'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['visibility'])

            element.append(json_pydata['data']['weather'][i]['hourly'][j]['pressure'])
            element.append(json_pydata['data']['weather'][i]['hourly'][j]['cloudcover'])
            elements.append(element)
       

GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-1-01&enddate=2013-1-28&includelocation=yes
GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-2-01&enddate=2013-2-28&includelocation=yes
GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-3-01&enddate=2013-3-28&includelocation=yes
GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-4-01&enddate=2013-4-28&includelocation=yes
GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-5-01&enddate=2013-5-28&includelocation=yes
GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358

thông tin api: https://www.worldweatheronline.com/developer/api/docs/historical-weather-api.aspx
các cột được chọn: 
Ngày + giờ
TempC : Nhiệt độ
Windspeed(km/h): tốc độ gió
Winddir16Point: hướng gió, trên la bàn 16 hướng
WeatherCode: thông tin thời tiết
Humidity
Visibility(km): tầm nhìn
Pressure: áp suất
Cloudcover: tỉ lệ che của mây

In [3]:
cols =['Day + Time','TempC','Windspeed(km/h)','Windir16Point','WeatherCode','Humidity (%)','Visibility (km)','Pressure (mb)','Cloudcover (%)']
data_df = pd.DataFrame(elements, columns=cols)

In [4]:
data_df

,Day + Time,TempC,Windspeed(km/h),Windir16Point,WeatherCode,Humidity (%),Visibility (km),Pressure (mb),Cloudcover (%)
0,2013-01-01 - 0,24,3,NNE,113,78,10,1011,20
1,2013-01-01 - 300,23,4,NNE,113,78,10,1010,22
2,2013-01-01 - 600,24,6,N,113,78,10,1011,21
3,2013-01-01 - 900,28,7,N,113,64,10,1012,16
4,2013-01-01 - 1200,32,3,NNW,113,52,10,1010,14
...,...,...,...,...,...,...,...,...,...
2074,2013-11-27 - 600,26,12,ENE,119,89,10,1009,33
2075,2013-11-27 - 900,30,14,E,113,70,10,1009,20
2076,2013-11-27 - 1200,32,12,ESE,116,59,10,1007,31
2077,2013-11-27 - 1500,30,11,ESE,356,72,7,1006,24


In [6]:
data_df.to_csv("rain.csv",index=False)
#data_df = pd.read_csv("rain.csv",index_col=None)

Chuyển các dữ liệu về dạng số 

In [32]:
def convert_col_dtype(col):
    if col.name not in ['Windir16Point', 'Day + Time']:
        return pd.to_numeric(col, errors='coerce')
    return col
data_df = data_df.apply(convert_col_dtype)

In [10]:
def missing_ratio(s):
    return s.isna().mean() * 100
def median(df):
    return df.quantile(0.5)
def lower_quartile(df):
    return df.quantile(0.25)
def upper_quartile(df):
    return df.quantile(0.75)

In [35]:
df = data_df.select_dtypes(exclude='object')
df.agg([missing_ratio, min, lower_quartile, median, upper_quartile, max])

,TempC,Windspeed(km/h),WeatherCode,Humidity (%),Visibility (km),Pressure (mb),Cloudcover (%)
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,21.0,0.0,0.0,28.0,0.0,1004.0,6.0
lower_quartile,25.0,6.0,0.0,63.5,10.0,1007.0,19.0
median,27.0,8.0,0.0,77.0,10.0,1009.0,29.0
upper_quartile,31.0,11.0,0.0,88.0,10.0,1010.0,45.0
max,38.0,23.0,1.0,98.0,10.0,1014.0,83.0


In [7]:
data_df['WeatherCode'].unique()

array(['113', '116', '176', '122', '266', '119', '296', '143', '293',
       '302', '353', '356', '299', '200', '386', '359', '263', '248',
       '389'], dtype=object)

Chuyển hướng gió từ 16 hướng -> 8 hướng? để dễ phân lớp?

Gán nhãn cho dữ liệu từ Weather code
https://gist.github.com/dawnvoh/a1f513123f7b6a42577d
    

In [16]:
#Những mã weather code của khi có mưa
rain_code = ['389','386','359','356','353','314','311','308','305','302','299','296','293','176']
data_df.loc[~data_df["WeatherCode"].isin(rain_code),"WeatherCode"] = "0"
data_df.loc[data_df["WeatherCode"].isin(rain_code),"WeatherCode"] = "1"

In [27]:
data_df["WeatherCode"].value_counts()

0    1771
1     308
Name: WeatherCode, dtype: int64

Split dữ liệu

In [ ]:
#Split dữ liệu

Transform các thứ các kiểu